We have fine-tuned FinBERT pretrained model on several financial NLP tasks, all outperforming traditional machine learning models, deep learning models, and fine-tuned BERT models.

All the fine-tuned FinBERT models are publicly hosted at Huggingface 🤗. Specifically, we have the following:
- **FinBERT-Sentiment**: for sentiment classification task
- **FinBERT-ESG**: for ESG classification task
- **FinBERT-FLS**: for forward-looking statement (FLS) classification task

*Note: the following code is for demonstration purpose. Please use GPU for fast inference on large scale dataset.*

In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

In [2]:
# tested in transformers==4.18.0 
import transformers
transformers.__version__

'4.46.2'

## Sentiment Analysis
Analyzing financial text sentiment is valuable as it can engage the views and opinions of managers, information intermediaries and investors. FinBERT-Sentiment is a FinBERT model fine-tuned on 10,000 manually annotated sentences from analyst reports of S&P 500 firms.

**Input**: A financial text.

**Output**: Positive, Neutral or Negative.

In [3]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [4]:
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
results = nlp(['growth is strong and we have plenty of liquidity.', 
               'there is a shortage of capital, and we need extra financing.',
              'formulation patents might protect Vasotec to a limited extent.'])

In [5]:
results

[{'label': 'Positive', 'score': 1.0},
 {'label': 'Negative', 'score': 0.9952379465103149},
 {'label': 'Neutral', 'score': 0.9979718327522278}]

In [44]:
import pandas as pd

STOCK = "AAPL"
YEARS = ["2022","2023"]

file = None
for year in YEARS:
    FILE='Datasets/News-'+STOCK+'-'+year+'.csv'
    ds=pd.read_csv(FILE, sep=',')
    file = pd.concat([file,ds])
file = file.sort_values(by='date', ascending=True)
file = file[file.relevance >0.7].reset_index(drop=True)
file


,date,title,summary,ticker,relevance,sentiment
0,20220312T010000,Apple Is at the Core of Software Company Jamf'...,"When you live by Apple, you die by Apple. Jamf...",AAPL,0.997247,-0.662196
1,20220314T070000,Buy This Software Firm's Stock. Apple Is at th...,Buy Jamf Stock. Apple Is at the Core of the So...,AAPL,0.997247,-0.662196
2,20220314T122305,US Warns China Against Helping Russia In Ukrai...,Russia sought China's military equipment to su...,AAPL,1.000000,-0.001084
3,20220314T155400,Apple Stock Alert: Will Foxconn Closure Sink i...,Apple stock is pulling back for the third stra...,AAPL,0.890807,0.013369
4,20220314T175436,An Apple iPhone Supplier's Factories Are Closi...,Apple Supplier's Factories Are Closing Due to ...,AAPL,0.990085,-0.177913
...,...,...,...,...,...,...
1734,20231228T203048,Joe Biden Vs. Apple Watch: White House Upholds...,One of the biggest stories in the technology s...,AAPL,0.717171,-0.035040
1735,20231229T044351,Apple Is 'Working Desperately' To Overcome Wat...,Apple Inc. AAPL is strategically seeking to ci...,AAPL,0.807885,-0.122074
1736,20231229T114500,2024 Could Spell Trouble for Warren Buffett an...,"Apple needs a strong 2024, or the stock could ...",AAPL,0.925690,0.111139
1737,20231229T203112,$100 Invested In This Stock 15 Years Ago Would...,Apple AAPL has outperformed the market over th...,AAPL,0.983605,0.387251


In [35]:
import numpy as np

headlines_array = np.array(file)
headlines_list = list(headlines_array[:,1])
headlines_list

["Apple Is at the Core of Software Company Jamf's Appeal",
 "Buy This Software Firm's Stock. Apple Is at the Core of Its Appeal.",
 'US Warns China Against Helping Russia In Ukraine War',
 'Apple Stock Alert: Will Foxconn Closure Sink iPhone Maker?',
 "An Apple iPhone Supplier's Factories Are Closing Because of Covid. Wall Street Isn't Worried, Yet.",
 "Is Now The Time To Buy Apple Stock? Kevin O'Leary Thinks So",
 'KeyBanc Remains Upbeat On Apple Citing iPhone SE 3; BofA, Piper Sandler See Limited Impact From China Lockdown',
 'Apple can withstand production disruptions in China, analysts say',
 'Why This Apple Analyst Is Pessimistic About The 2022 Mobile Market',
 'Apple  ( AAPL )  is Expected To Launch iPhone 14 in Four Variants',
 'Apple Analyst: Apple Car Team Dissolved For Now, With Quick Reorganization Critical To Meet 2025 Deadline',
 'Apple iPhones Led Smartphone Sales As 5G Handsets Formed Over 50% Of Sales',
 'Apple  ( AAPL )  Releases watchOS 8.5, Leads Smartwatch Market',


In [36]:
results =nlp(headlines_list)
len(results)

1739

In [37]:
pdr = pd.DataFrame(results)
pdr

,label,score
0,Neutral,0.990195
1,Positive,0.999999
2,Negative,0.999526
3,Neutral,0.999497
4,Neutral,0.729375
...,...,...
1734,Neutral,0.999925
1735,Neutral,0.970055
1736,Negative,0.999999
1737,Neutral,0.999997


In [47]:
file["FinBertScore"]= pdr["score"]
file["FinBertLabel"]= pdr["label"]
file

,date,title,summary,ticker,relevance,sentiment,score,FinBertScore,FinBertLabel
0,20220312T010000,Apple Is at the Core of Software Company Jamf'...,"When you live by Apple, you die by Apple. Jamf...",AAPL,0.997247,-0.662196,0.990195,0.990195,Neutral
1,20220314T070000,Buy This Software Firm's Stock. Apple Is at th...,Buy Jamf Stock. Apple Is at the Core of the So...,AAPL,0.997247,-0.662196,0.999999,0.999999,Positive
2,20220314T122305,US Warns China Against Helping Russia In Ukrai...,Russia sought China's military equipment to su...,AAPL,1.000000,-0.001084,0.999526,0.999526,Negative
3,20220314T155400,Apple Stock Alert: Will Foxconn Closure Sink i...,Apple stock is pulling back for the third stra...,AAPL,0.890807,0.013369,0.999497,0.999497,Neutral
4,20220314T175436,An Apple iPhone Supplier's Factories Are Closi...,Apple Supplier's Factories Are Closing Due to ...,AAPL,0.990085,-0.177913,0.729375,0.729375,Neutral
...,...,...,...,...,...,...,...,...,...
1734,20231228T203048,Joe Biden Vs. Apple Watch: White House Upholds...,One of the biggest stories in the technology s...,AAPL,0.717171,-0.035040,0.999925,0.999925,Neutral
1735,20231229T044351,Apple Is 'Working Desperately' To Overcome Wat...,Apple Inc. AAPL is strategically seeking to ci...,AAPL,0.807885,-0.122074,0.970055,0.970055,Neutral
1736,20231229T114500,2024 Could Spell Trouble for Warren Buffett an...,"Apple needs a strong 2024, or the stock could ...",AAPL,0.925690,0.111139,0.999999,0.999999,Negative
1737,20231229T203112,$100 Invested In This Stock 15 Years Ago Would...,Apple AAPL has outperformed the market over th...,AAPL,0.983605,0.387251,0.999997,0.999997,Neutral
